In [1]:
# Main
import os
import gc
import shutil
import time
import random
import cv2
import numpy as np 
import pandas as pd
import glob
from tqdm import tqdm
tqdm.pandas()
import re

# Data Visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image, display

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18

import string
from tqdm.notebook import tqdm
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import multiprocessing as mp


In [2]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

# list_sub = ["ccpd_base", "ccpd_fn", "ccpd_db", "ccpd_rotate", "ccpd_weather", "ccpd_blur"]
list_sub = ["ccpd_weather"]
BASE_PATH = "C:/Users/User/DeepLearning/Deep_Learning/final_project"

In [3]:
img_path = []
left_up_x = []
left_up_y = []
right_down_x = []
right_down_y = []
decoded_plate = []
location = []
license_images = []
for sub in list_sub:
    sub_path = os.path.join(BASE_PATH, sub)
    sub_files = glob.glob(os.path.join(sub_path, "*"))

    for file in sub_files:

        img_path.append(file)

        file_no_jpg = file.split("/")[-1][:-4]

        boundind_box = file_no_jpg.split("-")[2]
        left_up_bbox = boundind_box.split("_")[0]
        right_down_bbox = boundind_box.split("_")[1]
        left_up_x .append(left_up_bbox.split(",")[0])
        left_up_y .append(left_up_bbox.split(",")[1])
        right_down_x .append(right_down_bbox.split(",")[0])
        right_down_y .append(right_down_bbox.split(",")[1])



        temp = []
        plate_number =  file_no_jpg.split("-")[-3]
        plate_number_split =  plate_number.split('_')
        city = provinces[int(plate_number_split[0])]
        alp = alphabets[int(plate_number_split[1])]
        temp.append(city)
        temp.append(alp)
        
        for i in range(len(plate_number_split)-2):
            temp.append(ads[int(plate_number_split[i+2])])
        k = "".join(temp)
        decoded_plate.append(k)


        # exver_img = cv2.imread(file)
        # vertices = file_no_jpg.split("-")[3]
        # split = vertices.split("_")

        # x= (split[0].split(','))
        # y= (split[1].split(','))
        # z= (split[2].split(','))
        # w= (split[3].split(','))
        # coordinates= [int(x[0]), int(x[1]),int(y[0]),int(y[1]),int(z[0]),int(z[1]),int(w[0]),int(w[1])]
        # location.append(coordinates)
        # p1 = np.float32([[coordinates[4], coordinates[5]],
        #                 [coordinates[6], coordinates[7]],
        #                 [coordinates[2], coordinates[3]], 
        #                 [coordinates[0], coordinates[1]]])
        # p2 = np.float32([[0, 0],[exver_img.shape[1],0],[0,exver_img.shape[0]],[exver_img.shape[1],exver_img.shape[0]]])
        # M = cv2.getPerspectiveTransform(p1, p2)
        # outimg = cv2.warpPerspective(exver_img, M, exver_img.shape[1::-1])
        # resized = cv2.resize(outimg, (120,24), interpolation = cv2.INTER_AREA)
        # no_chinese_img = resized[:,width//8:]
        # license_images.append(resized)
        


        

r = [img_path,left_up_x,left_up_y,right_down_x,right_down_y,decoded_plate]
all_data = pd.DataFrame(r)
all_data = all_data.T
all_data.columns = ['img_path','x1','y1','x2','y2','decode_plate']
all_data                

,img_path,x1,y1,x2,y2,decode_plate
0,C:/Users/User/DeepLearning/Deep_Learning/final...,294,496,374,540,皖AD130W
1,C:/Users/User/DeepLearning/Deep_Learning/final...,304,542,411,577,皖AUT267
2,C:/Users/User/DeepLearning/Deep_Learning/final...,339,547,444,583,皖MZ4882
3,C:/Users/User/DeepLearning/Deep_Learning/final...,177,511,268,553,皖AVD028
4,C:/Users/User/DeepLearning/Deep_Learning/final...,402,426,497,467,皖RL222P
...,...,...,...,...,...,...
9994,C:/Users/User/DeepLearning/Deep_Learning/final...,89,832,500,1159,皖AK927W
9995,C:/Users/User/DeepLearning/Deep_Learning/final...,81,357,572,632,皖A22T99
9996,C:/Users/User/DeepLearning/Deep_Learning/final...,245,391,703,692,皖NRX399
9997,C:/Users/User/DeepLearning/Deep_Learning/final...,69,394,681,620,皖EW666K


In [4]:
list_split = ["train"] * 9499
for i in range(5):
    list_split.extend(["val"]*50)
    list_split.extend(["test"]*50)
all_data ["split"] = list_split
all_data 

,img_path,x1,y1,x2,y2,decode_plate,split
0,C:/Users/User/DeepLearning/Deep_Learning/final...,294,496,374,540,皖AD130W,train
1,C:/Users/User/DeepLearning/Deep_Learning/final...,304,542,411,577,皖AUT267,train
2,C:/Users/User/DeepLearning/Deep_Learning/final...,339,547,444,583,皖MZ4882,train
3,C:/Users/User/DeepLearning/Deep_Learning/final...,177,511,268,553,皖AVD028,train
4,C:/Users/User/DeepLearning/Deep_Learning/final...,402,426,497,467,皖RL222P,train
...,...,...,...,...,...,...,...
9994,C:/Users/User/DeepLearning/Deep_Learning/final...,89,832,500,1159,皖AK927W,test
9995,C:/Users/User/DeepLearning/Deep_Learning/final...,81,357,572,632,皖A22T99,test
9996,C:/Users/User/DeepLearning/Deep_Learning/final...,245,391,703,692,皖NRX399,test
9997,C:/Users/User/DeepLearning/Deep_Learning/final...,69,394,681,620,皖EW666K,test


In [5]:
no_chinese = []
for i in range(9999):
    no_chinese.append(all_data['decode_plate'][i][1:])

all_data['no_chinese'] = no_chinese
all_data 

,img_path,x1,y1,x2,y2,decode_plate,split,no_chinese
0,C:/Users/User/DeepLearning/Deep_Learning/final...,294,496,374,540,皖AD130W,train,AD130W
1,C:/Users/User/DeepLearning/Deep_Learning/final...,304,542,411,577,皖AUT267,train,AUT267
2,C:/Users/User/DeepLearning/Deep_Learning/final...,339,547,444,583,皖MZ4882,train,MZ4882
3,C:/Users/User/DeepLearning/Deep_Learning/final...,177,511,268,553,皖AVD028,train,AVD028
4,C:/Users/User/DeepLearning/Deep_Learning/final...,402,426,497,467,皖RL222P,train,RL222P
...,...,...,...,...,...,...,...,...
9994,C:/Users/User/DeepLearning/Deep_Learning/final...,89,832,500,1159,皖AK927W,test,AK927W
9995,C:/Users/User/DeepLearning/Deep_Learning/final...,81,357,572,632,皖A22T99,test,A22T99
9996,C:/Users/User/DeepLearning/Deep_Learning/final...,245,391,703,692,皖NRX399,test,NRX399
9997,C:/Users/User/DeepLearning/Deep_Learning/final...,69,394,681,620,皖EW666K,test,EW666K


In [6]:
df_train = all_data [all_data["split"]=="train"].reset_index(drop=True)
df_train = df_train[["img_path", "x1", "y1", "x2", "y2","decode_plate",'no_chinese']]
df_train

,img_path,x1,y1,x2,y2,decode_plate,no_chinese
0,C:/Users/User/DeepLearning/Deep_Learning/final...,294,496,374,540,皖AD130W,AD130W
1,C:/Users/User/DeepLearning/Deep_Learning/final...,304,542,411,577,皖AUT267,AUT267
2,C:/Users/User/DeepLearning/Deep_Learning/final...,339,547,444,583,皖MZ4882,MZ4882
3,C:/Users/User/DeepLearning/Deep_Learning/final...,177,511,268,553,皖AVD028,AVD028
4,C:/Users/User/DeepLearning/Deep_Learning/final...,402,426,497,467,皖RL222P,RL222P
...,...,...,...,...,...,...,...
9494,C:/Users/User/DeepLearning/Deep_Learning/final...,179,440,595,575,皖AR8647,AR8647
9495,C:/Users/User/DeepLearning/Deep_Learning/final...,190,506,572,653,皖A47039,A47039
9496,C:/Users/User/DeepLearning/Deep_Learning/final...,125,405,526,545,皖A4U877,A4U877
9497,C:/Users/User/DeepLearning/Deep_Learning/final...,290,482,680,626,皖AH790W,AH790W


In [7]:
df_val = all_data [all_data["split"]=="val"].reset_index(drop=True)
df_val = df_val[["img_path", "x1", "y1", "x2", "y2","decode_plate",'no_chinese']]
df_val

,img_path,x1,y1,x2,y2,decode_plate,no_chinese
0,C:/Users/User/DeepLearning/Deep_Learning/final...,217,448,475,666,皖AY6F46,AY6F46
1,C:/Users/User/DeepLearning/Deep_Learning/final...,134,424,504,576,浙C096FU,C096FU
2,C:/Users/User/DeepLearning/Deep_Learning/final...,54,394,380,567,沪C0C9U1,C0C9U1
3,C:/Users/User/DeepLearning/Deep_Learning/final...,281,536,662,684,皖AY060V,AY060V
4,C:/Users/User/DeepLearning/Deep_Learning/final...,162,445,586,578,皖ADB398,ADB398
...,...,...,...,...,...,...,...
245,C:/Users/User/DeepLearning/Deep_Learning/final...,147,426,643,595,皖AL1U67,AL1U67
246,C:/Users/User/DeepLearning/Deep_Learning/final...,199,362,533,613,皖AUL132,AUL132
247,C:/Users/User/DeepLearning/Deep_Learning/final...,233,338,589,574,皖AFJ956,AFJ956
248,C:/Users/User/DeepLearning/Deep_Learning/final...,94,424,601,592,皖AS661Z,AS661Z


In [19]:
from numba import cuda
from GPUtil import showUtilization as gpu_usage

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  3% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  3% |


In [20]:

class Dataset(Dataset):
    def __init__(self, dataframe):
        self.df_train = dataframe

    def __len__(self):
        return len(self.df_train)

    def __getitem__(self, index):
        train_path = self.df_train["img_path"][index]
        file_no_jpg = train_path.split("/")[-1][:-4]

        exver_img = cv2.imread(train_path)
        vertices = file_no_jpg.split("-")[3]
        split = vertices.split("_")

        x= (split[0].split(','))
        y= (split[1].split(','))
        z= (split[2].split(','))
        w= (split[3].split(','))
        coordinates= [int(x[0]), int(x[1]),int(y[0]),int(y[1]),int(z[0]),int(z[1]),int(w[0]),int(w[1])]
        location.append(coordinates)
        p1 = np.float32([[coordinates[4], coordinates[5]],
                        [coordinates[6], coordinates[7]],
                        [coordinates[2], coordinates[3]], 
                        [coordinates[0], coordinates[1]]])
        p2 = np.float32([[0, 0],[exver_img.shape[1],0],[0,exver_img.shape[0]],[exver_img.shape[1],exver_img.shape[0]]])
        M = cv2.getPerspectiveTransform(p1, p2)
        outimg = cv2.warpPerspective(exver_img, M, exver_img.shape[1::-1])
        resized = cv2.resize(outimg, (120,24), interpolation = cv2.INTER_AREA)
        
        # no_chinese_img = resized[:,120//8:]

        text = self.df_train["no_chinese"][index]

        # Apply transformations
        transformed_image = self.transform(resized)

        return transformed_image, text

    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((50, 200)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

In [13]:
trainset = Dataset(df_train)
valset =  Dataset(df_val)

train_loader = DataLoader(trainset, batch_size=16, num_workers=0, shuffle=False)
test_loader = DataLoader(valset, batch_size=16, num_workers=0, shuffle=False)
print(len(train_loader))

594


In [14]:
image_batch, text_batch = next(iter(train_loader))
print(image_batch.size(), text_batch)

torch.Size([16, 3, 50, 200]) ('AD130W', 'AUT267', 'MZ4882', 'AVD028', 'RL222P', 'AGC276', 'AU0194', 'AC880Y', 'ATG568', 'A41476', 'AH8V95', 'ATV235', 'AJ7532', 'AE0901', 'QSW808', 'A93853')


In [15]:
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O','0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
idx2char = {k:v for k,v in enumerate(alphabets, start=0)}
print(idx2char)
char2idx = {v:k for k,v in idx2char.items()}
print(char2idx)

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'J', 9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'P', 14: 'Q', 15: 'R', 16: 'S', 17: 'T', 18: 'U', 19: 'V', 20: 'W', 21: 'X', 22: 'Y', 23: 'Z', 24: 'O', 25: '0', 26: '1', 27: '2', 28: '3', 29: '4', 30: '5', 31: '6', 32: '7', 33: '8', 34: '9'}
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'J': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'U': 18, 'V': 19, 'W': 20, 'X': 21, 'Y': 22, 'Z': 23, 'O': 24, '0': 25, '1': 26, '2': 27, '3': 28, '4': 29, '5': 30, '6': 31, '7': 32, '8': 33, '9': 34}


In [16]:
num_chars = len(char2idx)
print(num_chars)
rnn_hidden_size = 256

35


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [17]:
resnet = resnet18(pretrained=True)

c:\Users\User\anaconda3\envs\DeepLearning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\envs\DeepLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class CRNN(nn.Module):
    
    def __init__(self, num_chars, rnn_hidden_size=256, dropout=0.4):
        
        super(CRNN, self).__init__()
        self.num_chars = num_chars
        self.rnn_hidden_size = rnn_hidden_size
        self.dropout = dropout
        
        # CNN Part 1
        resnet_modules = list(resnet.children())[:-3]
        self.cnn_p1 = nn.Sequential(*resnet_modules)
        
        # CNN Part 2
        self.cnn_p2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=(3,6), stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.linear1 = nn.Linear(1024, 256)
        
        # RNN
        self.rnn1 = nn.GRU(input_size=rnn_hidden_size, 
                            hidden_size=rnn_hidden_size,
                            bidirectional=True, 
                            batch_first=True)
        self.rnn2 = nn.GRU(input_size=rnn_hidden_size, 
                            hidden_size=rnn_hidden_size,
                            bidirectional=True, 
                            batch_first=True)
        self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)
        
        
    def forward(self, batch):
        
        batch = self.cnn_p1(batch)
        # print(batch.shape)
        
        batch = self.cnn_p2(batch)
        
        
        batch = batch.permute(0, 3, 1, 2) 
         
        batch_size = batch.size(0)
        T = batch.size(1)
        batch = batch.view(batch_size, T, -1) 
        batch = self.linear1(batch)
        
        
        batch, hidden = self.rnn1(batch)
        feature_size = batch.size(2)
        batch = batch[:, :, :feature_size//2] + batch[:, :, feature_size//2:]
       
        batch, hidden = self.rnn2(batch)
        
        
        batch = self.linear2(batch)
        # print(batch.shape)
        
        
        batch = batch.permute(1, 0, 2)
        # print(batch.shape)
        return batch
        
def weights_init(m):
    classname = m.__class__.__name__
    if type(m) in [nn.Linear, nn.Conv2d, nn.Conv1d]:
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

crnn = CRNN(num_chars, rnn_hidden_size=rnn_hidden_size)
crnn.apply(weights_init)
crnn = crnn.to(device)            

cuda


In [24]:
text_batch_logits = crnn(image_batch.to(device))
print(text_batch)
print(text_batch_logits.shape)

('AD130W', 'AUT267', 'MZ4882', 'AVD028', 'RL222P', 'AGC276', 'AU0194', 'AC880Y', 'ATG568', 'A41476', 'AH8V95', 'ATV235', 'AJ7532', 'AE0901', 'QSW808', 'A93853')
torch.Size([10, 16, 35])


In [25]:
criterion = nn.CTCLoss(blank=0)

In [28]:
def encode_text_batch(text_batch):
    
    text_batch_targets_lens = [len(text) for text in text_batch]
    text_batch_targets_lens = torch.IntTensor(text_batch_targets_lens)
    
    text_batch_concat = "".join(text_batch)
    text_batch_targets = [char2idx[c] for c in text_batch_concat]
    text_batch_targets = torch.IntTensor(text_batch_targets)
    
    return text_batch_targets, text_batch_targets_lens
def compute_loss(text_batch, text_batch_logits):
    """
    text_batch: list of strings of length equal to batch size
    text_batch_logits: Tensor of size([T, batch_size, num_classes])
    """
    text_batch_logps = F.log_softmax(text_batch_logits, 2) # [T, batch_size, num_classes]  
    text_batch_logps_lens = torch.full(size=(text_batch_logps.size(1),), 
                                       fill_value=text_batch_logps.size(0), 
                                       dtype=torch.int32).to(device) # [batch_size] 
    
    text_batch_targets, text_batch_targets_lens = encode_text_batch(text_batch)
    
    loss = criterion(text_batch_logps, text_batch_targets, text_batch_logps_lens, text_batch_targets_lens)

    return loss

compute_loss(text_batch, text_batch_logits)

tensor(4.9513, device='cuda:0', grad_fn=<MeanBackward0>)

In [27]:
num_epochs = 50
lr = 0.001
weight_decay = 1e-3
clip_norm = 5

optimizer = optim.Adam(crnn.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=5)


crnn = CRNN(num_chars, rnn_hidden_size=rnn_hidden_size)
crnn.apply(weights_init)
crnn = crnn.to(device)

In [29]:
epoch_losses = []
iteration_losses = []
num_updates_epochs = []
for epoch in tqdm(range(1, num_epochs+1)):
    epoch_loss_list = [] 
    num_updates_epoch = 0
    for image_batch, text_batch in tqdm(train_loader, leave=False):
        optimizer.zero_grad()
        text_batch_logits = crnn(image_batch.to(device))
        loss = compute_loss(text_batch, text_batch_logits)
        iteration_loss = loss.item()

        if np.isnan(iteration_loss) or np.isinf(iteration_loss):
            continue
          
        num_updates_epoch += 1
        iteration_losses.append(iteration_loss)
        epoch_loss_list.append(iteration_loss)
        loss.backward()
        nn.utils.clip_grad_norm_(crnn.parameters(), clip_norm)
        optimizer.step()

    epoch_loss = np.mean(epoch_loss_list)
    print("Epoch:{}    Loss:{}    NumUpdates:{}".format(epoch, epoch_loss, num_updates_epoch))
    epoch_losses.append(epoch_loss)
    num_updates_epochs.append(num_updates_epoch)
    lr_scheduler.step(epoch_loss)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/594 [00:00<?, ?it/s]

Epoch:1    Loss:1.7415344512422477    NumUpdates:594


  0%|          | 0/594 [00:00<?, ?it/s]

Epoch:2    Loss:1.2344428340995353    NumUpdates:594


...

Epoch:48    Loss:1.3094508870683534    NumUpdates:594


  0%|          | 0/594 [00:00<?, ?it/s]

Epoch:49    Loss:1.3094502731605813    NumUpdates:594


  0%|          | 0/594 [00:00<?, ?it/s]

Epoch:50    Loss:1.309449386516404    NumUpdates:594


In [30]:
torch.save(crnn.state_dict(), 'crnn_no_chinese_model_2.pth')

In [31]:
model = CRNN(num_chars=len(char2idx), rnn_hidden_size=256, dropout=0.1)

model.load_state_dict(torch.load('crnn_no_chinese_model.pth'))
model.to('cpu')

CRNN(
  (cnn_p1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [32]:
def decode_predictions(text_batch_logits):

    text_batch_tokens = F.softmax(text_batch_logits, 2).argmax(2) # [T, batch_size]
    text_batch_tokens = text_batch_tokens.numpy().T # [batch_size, T]

    text_batch_tokens_new = []
    for text_tokens in text_batch_tokens:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new

def remove_duplicates(text):
    if len(text) > 1:
        letters = [text[0]] + [letter for idx, letter in enumerate(text[1:], start=1) if text[idx] != text[idx-1]]
    elif len(text) == 1:
        letters = [text[0]]
    else:
        return ""
    return "".join(letters)

def correct_prediction(word):
    parts = word.split("-")
    parts = [remove_duplicates(part) for part in parts]
    corrected_word = "".join(parts)
    return corrected_word

results_val = pd.DataFrame(columns=['actual', 'prediction'])
val_loader = DataLoader(valset, batch_size=16, num_workers=0, shuffle=False)
with torch.no_grad():
    for image_batch, text_batch in tqdm(val_loader, leave=True):
        text_batch_logits = model(image_batch) # [T, batch_size, num_classes==num_features]
        text_batch_pred = decode_predictions(text_batch_logits.cpu())
        #print(text_batch, text_batch_pred)
        df = pd.DataFrame(columns=['actual', 'prediction'])
        df['actual'] = text_batch
        df['prediction'] = text_batch_pred
        results_val = pd.concat([results_val, df])
results_val = results_val.reset_index(drop=True)
results_val['prediction_corrected'] = results_val['prediction'].apply(correct_prediction)
results_val.head()

  0%|          | 0/16 [00:00<?, ?it/s]

,actual,prediction,prediction_corrected
0,AY6F46,YYYY664446,Y646
1,C096FU,0009996FFU,096FU
2,C0C9U1,CC00CC9911,C0C91
3,AY060V,YYYY00000V,Y0V
4,ADB398,DDDBBBB998,DB98


In [33]:
train_accuracy = accuracy_score(results_val['actual'], results_val['prediction_corrected'])
print(train_accuracy)

0.02


In [34]:
results_val['prediction_corrected']

0        Y646
1       096FU
2       C0C91
3         Y0V
4        DB98
        ...  
245      L167
246      UL12
247     FJ956
248      S61Z
249    BRTW97
Name: prediction_corrected, Length: 250, dtype: object

In [127]:
if (results_val['prediction_corrected'][0][0] == results_val['actual'][0][1]):
    print("yes")

yes


In [35]:
count = 0
for i in range(250):
    for j in range(6):
    
        for k in range(len(results_val['prediction_corrected'][i])):
        
            if(results_val['actual'][i][j] == results_val['prediction_corrected'][i][k]):
                count = count + 1
print(count/1750)                


0.7491428571428571
